# Upload Exported Records

This notebook reads the contents of an exported data set from the Test Monitor and uploads those records to another Test Monitor.

In [1]:
## Import Python Modules and Script Libraries

In [2]:
import os
import systemlink.clients.nifile as nifile
import systemlink.clients.nitestmonitor as nitestmonitor
script_file_ids = ["529ee0d2-bdba-4c27-9e98-7cf65682c379", "ab128c42-e281-4cd1-b974-018009a7d379"]

# Dynamically add the 2 python script libraries added previously to the File Service
connect_api = nifile.ApiClient()
files_api = nifile.FilesApi(connect_api)
for script_file_id in script_file_ids:
    response = await files_api.receive_file(script_file_id, _preload_content=False)
    print(str(response))
    exec(await response.read())

<ClientResponse(https://test-api.lifecyclesolutions.ni.com/nifile/v1/service-groups/Default/files/529ee0d2-bdba-4c27-9e98-7cf65682c379/data) [200 OK]>
<CIMultiDictProxy('Date': 'Sat, 12 Nov 2022 15:51:23 GMT', 'Content-Type': 'application/octet-stream', 'Content-Length': '8419', 'Connection': 'keep-alive', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Content-Disposition': 'attachment; filename="parse_exported_records.py"', 'Server': 'Cowboy', 'x-request-id': '2sje2aif6vi2h61ldg05nab1')>

<ClientResponse(https://test-api.lifecyclesolutions.ni.com/nifile/v1/service-groups/Default/files/ab128c42-e281-4cd1-b974-018009a7d379/data) [200 OK]>
<CIMultiDictProxy('Date': 'Sat, 12 Nov 2022 15:51:24 GMT', 'Content-Type': 'application/octet-stream', 'Content-Length': '4077', 'Connection': 'keep-alive', 'Cache-Control': 'max-age=0, private, must-revalidate', 'Content-Disposition': 'attachment; filename="upload_test_records.py"', 'Server': 'Cowboy', 'x-request-id': '2sje2ajaurklbba8no05nd

## Assign Authentication Parameters

In [3]:
host_url = os.getenv("SYSTEMLINK_HTTP_URI")
api_key = None # os.getenv("SYSTEMLINK_API_KEY")
workspace = "5e24401b-fdb6-4185-8364-723164504590" #  workspaceId for "TSE"
username = None
password = None

## Establish SystemLink Connection and Data Set Info

In [4]:
# Establish the connection to SystemLink in order to upload test records
upload_test_records = UploadTestRecords(host_url, api_key, username, password)

# Set the global data set loading details
data_file_dir = os.getcwd() + "/"
results_sheet_name = "TestResults"
steps_sheet_name = "TestSteps"
shift_datetime = True # Make all data sets appear to have occurred in the last 30 days since running this script
load_data_sets = [5] # [0, 1, 2, 3, 4, 5] # option to load a subset of the data sets 0..5

## Download the Desired Posted Data File

In [5]:
async def download_data_file(data_file_name, data_file_id):
    response = await files_api.receive_file(data_file_id, _preload_content=False)
    downloaded_file = open(data_file_name, "wb")
    resp = downloaded_file.write(await response.read())
    downloaded_file.close()

## Parse Data Set 0 and Upload Test Records

In [6]:
if 0 in load_data_sets:
    # Download the data files to the JupyterHub home directory
    await download_data_file("0 - Exported Results.csv", "8d86aadc-a9da-40be-bfa2-e431ae6ff9a5")
    await download_data_file("0 - Exported Steps.csv", "f91bb81e-61ab-404d-8315-a1b661a5439b")
    
    # The exported Mobile Phones Family data files have the following metadata column names
    keywords = ["DataSet0"]
    property_names = ["DOORS_ID", "Jira_ID", "Score_Count", "Score_Pct", "Team", "Temp~MaxAll", "Temp~MeanAll", "Temp~MinAll", "Temperature", "Test~Procedure", "Voltage"]
    input_names = ["Input.Team", "Input.Temperature", "Input.Voltage"]
    output_names = ["Output.Score_Pct"]

    # Parse all Battery Data from CSV files into array of results and dictionary of result_steps
    prefix = "Data Set 0 of 5 | "
    file_path_Results = data_file_dir + "0 - Exported Results.csv"
    file_path_Steps = data_file_dir + "0 - Exported Steps.csv"
    file_converter = FileConverter(prefix, "", file_path_Results, file_path_Steps, workspace, keywords, property_names, input_names, output_names, shift_datetime)

    # Upload/publish the results and their result_steps to the Test Monitor database, linked to the correct product, via web service API calls
    product = nitestmonitor.ProductRequestObject(part_number=None, name="MobileAB", family="Mobile Phones")
    await upload_test_records.upload(product, file_converter.results, file_converter.result_steps_dict, prefix)

## Parse Data Set 1 and Upload Test Records

In [7]:
if 1 in load_data_sets:
    # Download the data file to the JupyterHub home directory and the xlrd module for Pandas.read_excel()
    !pip install xlrd
    await download_data_file("1 - Exported Battery Data1.xls", "ad65d871-c141-4602-8db7-7509a4b074c3")
    
    # The exported AA Alkaline Family data files have the following metadata column names
    keywords = ["DataSet1"]
    property_names = ["Capacity (mAh)", "Composition", "Location", "nitmProcessModel", "nitmSource", "nitmTestSocketCount", "nitmTestStandStartTime", "Voltage (V)"]
    input_names = ["Input.Cycle", "Input.Initial Capacity  (mAh)", "Input.Voltage"]
    output_names = []
    
    # Parse the Battery Data1 Excel file into array of results and dictionary of result_steps
    prefix = "Data Set 1 of 5 | "
    file_path = data_file_dir + "1 - Exported Battery Data1.xls"
    file_converter = FileConverter(prefix, file_path, results_sheet_name, steps_sheet_name, workspace, keywords, property_names, input_names, output_names, shift_datetime)
    
    # Upload/publish the results and their result_steps to the Test Monitor, linked to the correct product
    product = nitestmonitor.ProductRequestObject(part_number=None, name="AA Alkaline", family="Batteries")
    await upload_test_records.upload(product, file_converter.results, file_converter.result_steps_dict, prefix)

## Parse Data Set 2 and Upload Test Records

In [8]:
if 2 in load_data_sets:
    # Download the data file to the JupyterHub home directory and the xlrd module for Pandas.read_excel()
    !pip install xlrd
    await download_data_file("2 - Exported Battery Data2.xls", "1d3a2892-de39-46b5-8381-75ff13775e02")
    
    # The exported AA Rechargable Family data files have the following metadata column names
    keywords = ["DataSet2"]
    property_names = ["Capacity (mAh)", "Composition", "Location", "nitmProcessModel", "nitmSource", "nitmTestSocketCount", "nitmTestStandStartTime", "Voltage (V)"]
    input_names = ["Input.Cycle", "Input.Initial Capacity  (mAh)", "Input.Voltage"]
    output_names = []
    
    # Parse the Battery Data2 Excel file into array of results and dictionary of result_steps
    prefix = "Data Set 2 of 5 | "
    file_path = data_file_dir + "2 - Exported Battery Data2.xls"
    file_converter = FileConverter(prefix, file_path, results_sheet_name, steps_sheet_name, workspace, keywords, property_names, input_names, output_names, shift_datetime)
    
    # Upload/publish the results and their result_steps to the Test Monitor, linked to the correct product
    product = nitestmonitor.ProductRequestObject(part_number=None, name="AA Rechargeable", family="Batteries")
    await upload_test_records.upload(product, file_converter.results, file_converter.result_steps_dict, prefix)

## Parse Data Set 3 and Upload Test Records

In [9]:
if 3 in load_data_sets:
    # Download the data file to the JupyterHub home directory and the xlrd module for Pandas.read_excel()
    !pip install xlrd
    await download_data_file("3 - Exported Mobile Data.xls", "c2032921-ebe3-4d05-9130-b5d278a101be")
    
    # The exported Transmitter data files have the following metadata column names
    keywords = ["DataSet3"]
    property_names = ["nitmProcessModel", "nitmSource", "nitmTestSocketCount", "nitmTestStandStartTime"]
    input_names = []
    output_names = []
    
    # Parse the Transmitter Excel file into array of results and dictionary of result_steps
    prefix = "Data Set 3 of 5 | "
    file_path = data_file_dir + "3 - Exported Mobile Data.xls"
    file_converter = FileConverter(prefix, file_path, results_sheet_name, steps_sheet_name, workspace, keywords, property_names, input_names, output_names, shift_datetime)
    
    # Upload/publish the results and their result_steps to the Test Monitor database, linked to the correct product, via web service API calls
    product = nitestmonitor.ProductRequestObject(part_number=None, name="Transmitter", family="RF")
    await upload_test_records.upload(product, file_converter.results, file_converter.result_steps_dict, prefix)

## Parse Data Set 4 and Upload Test Records

In [10]:
if 4 in load_data_sets:
    # Download the data file to the JupyterHub home directory and the xlrd module for Pandas.read_excel()
    !pip install xlrd
    await download_data_file("4 - Exported NI 5678 Data.xls", "58711e12-35ea-4c89-895d-0a7774cb58ae")
    
    # The exported NI Family data files have the following metadata column names
    keywords = ["DataSet4"]
    property_names = ["jobName", "location", "opSeq", "opSeqName", "tdms_file", "tdr_xml_doc_guid", "testDomain", "toolkitVersion"]
    input_names = []
    output_names = ["Output.Harmonics results~Frequency (Hz)", "Output.Output Power Results~Measured Output power (dBm)", "Output.Pass"]
    
    # Parse the NI Family Excel file into array of results and dictionary of result_steps
    prefix = "Data Set 4 of 5 | "
    file_path = data_file_dir + "4 - Exported NI 5678 Data.xls"
    file_converter = FileConverter(prefix, file_path, results_sheet_name, steps_sheet_name, workspace, keywords, property_names, input_names, output_names, shift_datetime)
    
    # Upload/publish the results and their result_steps to the Test Monitor database, linked to the correct product, via web service API calls
    product = nitestmonitor.ProductRequestObject(part_number=None, name="NI 5678", family="NI")
    await upload_test_records.upload(product, file_converter.results, file_converter.result_steps_dict, prefix)


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Data Set 4 of 5 | Reading Test Results and Test Steps from the data file...
Data Set 4 of 5 | Reading Test Result 2 of 313 (1.0%)
Data Set 4 of 5 | Reading Test Result 5 of 313 (2.0%)
Data Set 4 of 5 | Reading Test Result 8 of 313 (3.0%)
Data Set 4 of 5 | Reading Test Result 11 of 313 (4.0%)
Data Set 4 of 5 | Reading Test Result 15 of 313 (5.0%)
Data Set 4 of 5 | Reading Test Result 18 of 313 (6.0%)
Data Set 4 of 5 | Reading Test Result 21 of 313 (7.0%)
Data Set 4 of 5 | Reading Test Result 24 of 313 (8.0%)
Data Set 4 of 5 | Reading Test Result 27 of 313 (9.0%)
Data Set 4 of 5 | Reading Test Result 30 of 313 (10.0%)
Data Set 4 of 5 | Reading Test Result 33 of 313 (11.0%)
Data Set 4 of 5 | Reading Test Result 36 of 313 (12.0%)
Data Set 4 of 5 | Reading Test Result 40 of 313 (13.0%)
Data Set 4 of 5 | Reading Test Result 43 of 313 (14.0%)
Data Set 4 of 5 | Reading Test Result 46 of 

## Parse Data Set 5 and Upload Test Records

In [11]:
if 5 in load_data_sets:
    # Download the data file to the JupyterHub home directory and the xlrd module for Pandas.read_excel()
    !pip install xlrd
    await download_data_file("5 - Exported Transmitter Data.xls", "3cf48281-4933-463b-8526-f0d116df276d")
    
    # The exported NI Family data files have the following metadata column names
    keywords = ["DataSet5"]
    property_names = ["TestFWVersion", "TestRev"]
    input_names = ["Input.Frequency (Hz)", "Input.FW Version", "Input.Software Revision", "Input.Testpoint", "Input.UUT Mode"]
    output_names = []
    
    # Parse the NI Family Excel file into array of results and dictionary of result_steps
    prefix = "Data Set 5 of 5 | "
    file_path = data_file_dir + "5 - Exported Transmitter Data.xls"
    file_converter = FileConverter(prefix, file_path, results_sheet_name, steps_sheet_name, workspace, keywords, property_names, input_names, output_names, shift_datetime)
    
    # Upload/publish the results and their result_steps to the Test Monitor database, linked to the correct product, via web service API calls
    product = nitestmonitor.ProductRequestObject(part_number=None, name="TX ABC123", family="Transmitter")
    await upload_test_records.upload(product, file_converter.results, file_converter.result_steps_dict, prefix)